## Library

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import *
import tensorflow as tf
import numpy as np
import pandas as pd
from gensim import models
from konlpy.tag import Mecab
import re

## word2vec model (300 dimension)

In [2]:
mecab_model = models.word2vec.Word2Vec.load('/Users/jaeyoungshin/Desktop/NLP/cnn_2조/mecab_model_files/word2vec_ modeling/NaverMovie_mecab300.model')

## train/test set

In [3]:
train_data = pd.read_csv('/Users/jaeyoungshin/Desktop/NLP/cnn_2조/prep for modeling/ratings_train.txt',sep='\t').dropna().reset_index(drop=True)
test_data = pd.read_csv('/Users/jaeyoungshin/Desktop/NLP/cnn_2조/prep for modeling/ratings_test.txt',sep='\t').dropna().reset_index(drop=True)

training_sentences = []
testing_sentences = []

In [4]:
stopwords_df = pd.read_excel('/Users/jaeyoungshin/Desktop/NLP/cnn_2조/prep for modeling/stopwords.xlsx')
stopwords_list = stopwords_df['불용어'].tolist()

In [5]:
mecab = Mecab()

In [6]:
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [7]:
documents = []
for item in train_data.document:
    documents.append([word for word in [tokens[0] for tokens in mecab.pos(item)] if word not in stopwords_list])

labels = []
for item in train_data.label:
    labels.append(item)

In [8]:
test_documents = []
for item in test_data.document:
    test_documents.append([word for word in [tokens[0] for tokens in mecab.pos(item)] if word not in stopwords_list])

test_labels = []
for item in test_data.label:
    test_labels.append(item)

In [9]:
training_labels_final = np.array(labels)
testing_labels_final = np.array(test_labels)

## Padding

In [10]:
vocab_size = 20000
embedding_dim = 300
max_length = 41
oov_tok = '<OOV>'

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(documents)

In [11]:
testing_sequences = tokenizer.texts_to_sequences(test_documents)
test_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [13]:
word_index = tokenizer.word_index
text_sequences = tokenizer.texts_to_sequences(documents)
padded = pad_sequences(text_sequences,maxlen=max_length,truncating = 'pre')

## save tokenizer as pickle file

In [14]:
import pickle

In [15]:
with open('tokenizer_mecab300_l2.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

## save parameters as json file

In [16]:
import json
from collections import OrderedDict

file_data = OrderedDict()
file_data['max_len'] = 41
file_data['pad_type'] = "pre"
file_data['trunc_type'] = "pre"

In [17]:
print(json.dumps(file_data, ensure_ascii=False, indent='\t'))

{
	"max_len": 41,
	"pad_type": "pre",
	"trunc_type": "pre"
}


In [18]:
with open('w2v_mecab300_l2.json','w',encoding='utf-8') as make_file:
    json.dump(file_data, make_file, ensure_ascii=False, indent='\t')

## Embedding Matrix

In [19]:
embedding_matrix = np.zeros((vocab_size, embedding_dim))

index = 0 
for word, idx in tokenizer.word_index.items():
    try:
        embedding_vector = mecab_model.wv.__getitem__(word)
        if embedding_vector is not None:
            embedding_matrix[idx] = embedding_vector
    except Exception as e:
        pass
        
embedding_matrix.shape

(20000, 300)

## Layers

In [20]:
sequence_input = tf.keras.Input(shape=(max_length,), dtype='int32')
embedding_layer = tf.keras.layers.Embedding(vocab_size,
                            embedding_dim,
                            input_length=max_length,
                            trainable=False)
embedded_sequences = embedding_layer(sequence_input)
convs = []
filter_sizes = [3,4,5]
for fsz in filter_sizes:
    x = tf.keras.layers.Conv1D(128, fsz, activation='relu',padding='same')(embedded_sequences)
    x = tf.keras.layers.MaxPooling1D()(x)
    convs.append(x)
x = tf.keras.layers.Concatenate(axis=-1)(convs)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.02))(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(sequence_input, output)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Training

In [21]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_accuracy',min_delta=0.002,patience=1,mode='max')

In [22]:
num_epochs = 30
history = model.fit(padded, training_labels_final, epochs = num_epochs, validation_data = (test_padded, testing_labels_final), callbacks=[es])
history

Epoch 1/30
4688/4688 [==============================] - 120s 26ms/step - loss: 0.5821 - accuracy: 0.7503 - val_loss: 0.5277 - val_accuracy: 0.7941
Epoch 2/30
4688/4688 [==============================] - 121s 26ms/step - loss: 0.5168 - accuracy: 0.8131 - val_loss: 0.5141 - val_accuracy: 0.8150
Epoch 3/30
4688/4688 [==============================] - 121s 26ms/step - loss: 0.5019 - accuracy: 0.8355 - val_loss: 0.5243 - val_accuracy: 0.8164


In [23]:
sentence = ["수면제인가 꿀잠잤음","바보같은 영화","조금 기대 이하이긴 했는데 그래도 만족함", "완전 재밌었다", "개꿀잼"]
sequence_exp = tokenizer.texts_to_sequences(sentence)
padded_exp = pad_sequences(sequence_exp, maxlen = max_length, truncating= 'post')
print(model.predict(padded_exp).round(2))

[[0.65]
 [0.52]
 [0.66]
 [0.64]
 [0.49]]


## Save model

In [24]:
model.save('word2vec_mecab300_updated_l2/0.8164_model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: word2vec_mecab300_updated_l2/0.8164_model/assets


**load saved model / re-training**

In [29]:
mecab_model = tf.keras.models.load_model('word2vec_mecab300_updated_l2/0.8164_model')

In [28]:
from keras.callbacks import EarlyStopping
re_es = EarlyStopping(monitor='val_accuracy',min_delta=0.002,mode='max')

In [30]:
re_history = mecab_model.fit(padded, training_labels_final, epochs = num_epochs, validation_data = (test_padded, testing_labels_final), callbacks=[re_es])
re_history

Epoch 1/30
4688/4688 [==============================] - 349s 75ms/step - loss: 0.5169 - accuracy: 0.8350 - val_loss: 0.4684 - val_accuracy: 0.8474
Epoch 2/30
4688/4688 [==============================] - 349s 74ms/step - loss: 0.4069 - accuracy: 0.8859 - val_loss: 0.4815 - val_accuracy: 0.8468


**re-training model save (val_accuracy : 0.8468)**

In [31]:
mecab_model.save('word2vec_mecab300_updated_l2/0.8468_model')

INFO:tensorflow:Assets written to: word2vec_mecab300_updated_l2/0.8468_model/assets


In [32]:
mecab_remodel = tf.keras.models.load_model('word2vec_mecab300_updated_l2/0.8468_model')

In [60]:
# 0.7993
sentence = ["완전 추천함","아 겁나 지루했음 개비추..","에라이 때려쳐 이게 영화라고?",'내 인생 영화']
sequence_exp = tokenizer.texts_to_sequences(sentence)
padded_exp = pad_sequences(sequence_exp, maxlen = max_length, truncating= 'post')
print(mecab_remodel.predict(padded_exp).round(2))

[[0.99]
 [0.05]
 [0.02]
 [0.76]]


In [41]:
# 0.8468
sentence = ["완전 추천함","아 겁나 지루했음 개비추..","에라이 때려쳐 이게 영화라고?",'내 인생 영화']
sequence_exp = tokenizer.texts_to_sequences(sentence)
padded_exp = pad_sequences(sequence_exp, maxlen = max_length, truncating= 'post')
print(mecab_remodel.predict(padded_exp).round(2))

[[0.57]
 [0.63]
 [0.08]
 [0.89]]
